In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage
import time
import numpy as np
import h5py
f = h5py.File("/nfs/chess/raw/2023-3/id3a/berman-3804-a/zx30-t1-ff-0/1/ff/ff1_000231.h5", 'r')
data=f["imageseries"]["images"][:,:,:]

# need for max filter (if I used uint16 i would need an extra bit)
data=data.astype(np.float32,copy=False) 

In [ ]:
max_projection=np.max(data,axis=0)

In [14]:
import tifffile
import numpy as np
import PIL

import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

# ////////////////////////////////////////////////
def ShowSaturation(img,user_zmin,user_zmax):

    H,W=img.shape
    zmin,zmax=np.min(img),np.max(img)

    show_min        = widgets.BoundedIntText(value=user_zmin, min=zmin, max=zmax, step=1, description=f'Min/{zmin}')
    show_max        = widgets.BoundedIntText(value=user_zmax, min=zmin, max=zmax, step=1, description=f'Max/{zmax}')
    show_saturation = widgets.Checkbox(description='Show saturation: ',value=True)

    fig = go.Figure()
    fig.layout.dragmode='pan'

    fig.add_trace(go.Scatter(x=[0, W ],y=[0, H],mode="markers",marker_opacity=0)) # This trace is added to help the autoresize logic work.
    fig.update_xaxes(visible=False,range=[0, W])
    fig.update_yaxes(visible=False,range=[0, H],scaleanchor="x")

    # ////////////////////////////////////////////////
    def CreateImage(r=None,g=None,b=None):
        if r is None:
            gray=(np.random.random([256,256])*255.0).astype(np.uint8)
            return CreateImage(gray,gray,gray)
        return PIL.Image.fromarray(np.stack([r,g,b],axis=-1))


    fig.add_layout_image(dict(
        x=0, y=H, sizex=W, sizey=H,
        xref="x",yref="y",opacity=1.0, layer="below",sizing="stretch",
        source=CreateImage()
    ))
    fig.update_layout(margin={"l": 0, "r": 0, "t": 0, "b": 0})

    figure_widget=go.FigureWidget(fig)
    figure_widget.set_trait("_config",{"scrollZoom": True})  # if you cannot debug, comment this line

    def onChange(change=None):
        print(f"RefreshColors min={show_min.value} max={show_max.value} show_staturation={show_saturation.value} ")
        r=g=b=(255.0*(img/zmax)).astype(np.uint8)
        if show_saturation.value: 
            g,b=np.copy(r),np.copy(r)
            mask=r>(255.0*(show_max.value/zmax))
            r[mask],g[mask],b[mask]=255,0,0
        new_image=CreateImage(r,g,b)
        figure_widget.update_layout_images(source=new_image)


    show_min       .observe(onChange, names="value")
    show_max       .observe(onChange, names="value")
    show_saturation.observe(onChange, names="value")

    onChange()
    return widgets.HBox([figure_widget,widgets.VBox([show_min,show_max,show_saturation])])

ShowSaturation(max_projection,200,15000)

RefreshColors min=285 max=15000 show_staturation=True 


    'data': [{'marker': {'opacity': 0},
              'mode': 'markers',
       …

Write the data without filters

In [15]:
import sys,os,shutil
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
os.environ["VISUS_DISABLE_WRITE_LOCK"]="1"
import OpenVisus as ov
D,H,W=data.shape

idx_filename='./tmp/visus.idx'
shutil.rmtree(os.path.splitext(idx_filename)[0], ignore_errors=True)
field=ov.Field.fromString("DATA float32 default_layout(row_major)")
db = ov.CreateIdx(url=idx_filename, dim=3, dims=(W,H,D), fields=[field], compression="raw") 
t1 = time.time()
db.write(data)
print(f"db.write done in {time.time()-t1} seconds")

db.write done in 142.13905215263367 seconds


In [16]:
t1 = time.time()
db.compressDataset(["zip"])
print(f"db.compressDataset done in {time.time()-t1} seconds")

db.compressDataset done in 527.7871034145355 seconds


Check the data is the same

In [17]:
ov_max_projection=db.read(z=[0,D])
ShowSaturation(ov_max_projection,200,15000)

Write the data with **MAX** filter

In [ ]:
import sys,os,shutil
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
os.environ["VISUS_DISABLE_WRITE_LOCK"]="1"
import OpenVisus as ov
D,H,W=data.shape

idx_filename='./tmp/visus-max.idx'
shutil.rmtree(os.path.splitext(idx_filename)[0], ignore_errors=True)
field=ov.Field.fromString("DATA float32 default_layout(row_major)")
db = ov.CreateIdx(url=idx_filename, dim=3, dims=(W,H,D), filters=['max'], fields=[field], compression="raw") 
db.write(data)

for img in db.read(z=[100,100+1],num_refinements=4):
    assert img.shape[0]==1
    print(img.shape,img.dtype)
    ShowData(img[0,:,:])

Compute the filter (SLOW)

In [ ]:
sliding_window=1024 # i.e. 1k cube of float32 == 4GB
db.computeFilter(db.getField(), sliding_window)
print("filter computed")

In [ ]:

for img in db.read(z=[100,100+1],num_refinements=4):
    assert img.shape[0]==1
    print(img.shape,img.dtype)
    ShowData(img[0,:,:])